<a href="https://colab.research.google.com/github/rmwvamp/ECGNotebooks/blob/main/ECGFeatureExtractionCode_CLASDataset_Allfiles_SingleCode_32Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drive Mounting and Dependencies



In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
%cd /gdrive/MyDrive/SamsungWork/Participant2Data

/gdrive/MyDrive/SamsungWork/Participant2Data


In [ ]:
!pip install wfdb
!pip install opensignalsreader
!pip install pyhrv
!pip install biosppy
!pip install pyhrv
!pip install hrv-analysis

     |████████████████████████████████| 137 kB 4.3 MB/s 
     |████████████████████████████████| 11.2 MB 30.6 MB/s 
     |████████████████████████████████| 930 kB 40.3 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 3.2 MB 3.3 MB/s 
     |████████████████████████████████| 1.5 MB 43.1 MB/s 
     |████████████████████████████████| 230 kB 61.6 MB/s 
     |████████████████████████████████| 47 kB 5.8 MB/s 
  Created wheel for biosppy: filename=biosppy-0.8.0-py2.py3-none-any.whl size=100176 sha256=9bd22eef4df6a4cd70d3e4af480dcd56e02e35953d1829735b1ddada4d8d51f3
  Stored in directory: /root/.cache/pip/wheels/32/17/9d/77adbf0db713546eaa1c7a1480b2dce91fa724a133115dc20c
  Created wheel for spectrum: filename=spectrum-0.8.1-cp37-cp37m-linux_x86_64.whl size=235159 sha256=588408d7672399b4d8ba50e3c0eddac85fe61637c3c7dae6c4b5964fe973f9a9
  Stored in directory: /root/.cache/pip/wheels/79/db/9c/92fa684ca088447807d08672e7609b48102c6161ac9c7e3c62
  Created wheel for easydev: filename=easydev-0.12.0-py3-none-any.whl size=64232 sha256=20fdca342595e0dc05886384d09dad182db5ce177c8ae1556bde6f7de6c98f47
  Stored in directory: /r

In [ ]:
import os
import sys
import csv

#median
import statistics
from statistics import median

import os
from glob import glob

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

# physionet data
import wfdb
from wfdb import processing

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',100000)

from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from hrvanalysis import *

# Feature Function Declaration

## Function for Logic Part

In [ ]:
def get_plot_ranges(start=0, end=150, n=2.5):
    '''
    Make an iterator that divides into n or n+1 ranges. 
    - if end-start is divisible by steps, return n ranges
    - if end-start is not divisible by steps, return n+1 ranges, where the last range is smaller and ends at n
    
    # Example:
    >> list(get_plot_ranges())
    >> [(0.0, 3.0), (3.0, 6.0), (6.0, 9.0)]

    '''
    distance = end - start
    for i in np.arange(start, end, np.floor(distance/n)):
        yield (int(i), int(np.minimum(end, np.floor(distance/n) + i)))

def group_peaks(p, threshold=5):
    '''
    The peak detection algorithm finds multiple peaks for each QRS complex. 
    Here we group collections of peaks that are very near (within threshold) and we take the median index 
    '''
    # initialize output
    output = np.empty(0)

    # label groups of sample that belong to the same peak
    peak_groups, num_groups = label(np.diff(p) < threshold)

    # iterate through groups and take the mean as peak index
    for i in np.unique(peak_groups)[1:]:
        peak_group = p[np.where(peak_groups == i)]
        output = np.append(output, np.median(peak_group))
    return output

def detect_peaks(ecg_signal, threshold=0.3, qrs_filter=None):
    '''
    Peak detection algorithm using cross corrrelation and threshold 
    '''
    if qrs_filter is None:
        # create default qrs filter
        t = np.linspace(1.5 * np.pi, 3.5 * np.pi, 15)
        qrs_filter = np.sin(t)
    
    # normalize data
    ecg_signal = (ecg_signal - ecg_signal.mean()) / ecg_signal.std()

    # calculate cross correlation
    similarity = np.correlate(ecg_signal, qrs_filter, mode="same")
    similarity = similarity / np.max(similarity)

    # return peaks (values in ms) using threshold
    return ecg_signal[similarity > threshold].index, similarity


def get_rr_corrected(df):
  # detect peaks
  peaks, similarity = detect_peaks(df.ecg2, threshold=0.3)

  # group peaks so we get a single peak per beat (hopefully)
  grouped_peaks = group_peaks(peaks)

  # RR-intervals are the differences between successive peaks
  rr = np.diff(grouped_peaks)

  rr_corrected = rr.copy()
  rr_corrected[np.abs(zscore(rr)) > 2] = np.median(rr)
  return rr_corrected

# def plot_rr_corrected(rr_corrected,rr):

#   plt.title("RR-intervals", fontsize=24)
#   plt.xlabel("Time (ms)", fontsize=16)
#   plt.ylabel("RR-interval (ms)", fontsize=16)

#   plt.plot(rr, color="red", linewidth=1, label="RR-intervals")
#   plt.plot(rr_corrected, color="green", linewidth=2, label="RR-intervals after correction")
#   plt.legend(fontsize=20)

# def timedomain(rr):

#   results = {}
  
#   hr = 60000/rr
  
#   # mean RR-interval
#   results['Mean RR (ms)'] = np.mean(rr)
#   results['STD RR/SDNN (ms)'] = np.std(rr)
#   results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
#   results['Mean HR (beats/min)'] = np.mean(hr)
#   results['STD HR (beats/min)'] = np.std(hr)
#   results['Min HR (beats/min)'] = np.min(hr)
#   results['Max HR (beats/min)'] = np.max(hr)
#   results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
#   results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
#   results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
#   # print("Time domain metrics - automatically corrected RR-intervals:")
#   # for k, v in results.items():
#   #     print("- %s: %.2f" % (k, v))
#   return results

# def frequency_domain(rr, fs=4):     #4

#   # sample rate for interpolation
#   steps = 1 / fs

#   # create interpolation function based on the rr-samples. 
#   x = np.cumsum(rr) / 1000.0
#   f = interp1d(x, rr, kind='cubic',fill_value="extrapolate")

#   # now we can sample from interpolation function
#   xx = np.arange(1, np.max(x), steps)
#   rr_interpolated = f(xx)
#   # Estimate the spectral density using Welch's method
#   fxx, pxx = signal.welch(x=rr_interpolated, fs=fs)
  
#   '''
#   Segement found frequencies in the bands 
#     - Very Low Frequency (VLF): 0-0.04Hz 
#     - Low Frequency (LF): 0.04-0.15Hz 
#     - High Frequency (HF): 0.15-0.4Hz
#   '''
#   cond_vlf = (fxx >= 0) & (fxx < 0.04)
#   cond_lf = (fxx >= 0.04) & (fxx < 0.15)
#   cond_hf = (fxx >= 0.15) & (fxx < 0.4)
  
#   # calculate power in each band by integrating the spectral density 
#   vlf = trapz(pxx[cond_vlf], fxx[cond_vlf])
#   lf = trapz(pxx[cond_lf], fxx[cond_lf])
#   hf = trapz(pxx[cond_hf], fxx[cond_hf])
  
#   # sum these up to get total power
#   total_power = vlf + lf + hf

#   # find which frequency has the most power in each band
#   peak_vlf = fxx[cond_vlf][np.argmax(pxx[cond_vlf])]
#   peak_lf = fxx[cond_lf][np.argmax(pxx[cond_lf])]
#   peak_hf = fxx[cond_hf][np.argmax(pxx[cond_hf])]

#   # fraction of lf and hf
#   lf_nu = 100 * lf / (lf + hf)
#   hf_nu = 100 * hf / (lf + hf)
  
#   results = {}
#   results['Power VLF (ms2)'] = vlf
#   results['Power LF (ms2)'] = lf
#   results['Power HF (ms2)'] = hf   
#   results['Power Total (ms2)'] = total_power

#   results['LF/HF'] = (lf/hf)
#   results['Peak VLF (Hz)'] = peak_vlf
#   results['Peak LF (Hz)'] = peak_lf
#   results['Peak HF (Hz)'] = peak_hf

#   results['Fraction LF (nu)'] = lf_nu
#   results['Fraction HF (nu)'] = hf_nu
  
#   # print("Frequency domain metrics:")
#   # for k, v in results.items():
#   #     print("- %s: %.2f" % (k, v))
#   return results, fxx, pxx
def get_rr_corrected_ourdata(df):
  # detect peaks
  t = np.linspace(1.5 * np.pi, 3.5 * np.pi,13)
  qrs_filter = -np.sin(t)
  peaks, similarity = detect_peaks(df.ecg2, threshold=0.03,qrs_filter=qrs_filter)
 
  # group peaks so we get a single peak per beat (hopefully)
  grouped_peaks = group_peaks(peaks,threshold=3)
  # print(grouped_peaks)

  final=[element * 65 for element in grouped_peaks] 
  # print(final)
  # RR-intervals are the differences between successive peaks
  rr = np.diff(final)

  rr_corrected = rr.copy()
  rr_corrected[np.abs(zscore(rr)) > 2] = np.median(rr)
  return rr_corrected




In [ ]:
def windowing(df,windowsizerow):
  # print("in function1")
  # column_names=['Mean RR (ms)', 'STD RR/SDNN (ms)', "Mean HR (Kubios' style) (beats/min)", 'Mean HR (beats/min)', 'STD HR (beats/min)', 'Min HR (beats/min)', 'Max HR (beats/min)', 'RMSSD (ms)', 'NNxx', 'pNNxx (%)', 'Power VLF (ms2)', 'Power LF (ms2)', 'Power HF (ms2)', 'Power Total (ms2)', 'LF/HF', 'Peak VLF (Hz)', 'Peak LF (Hz)', 'Peak HF (Hz)', 'Fraction LF (nu)', 'Fraction HF (nu)']
  column_names=['mean_nni','sdnn','sdsd','rmssd','median_nni','nni_50','pnni_50','nni_20','pnni_20','range_nni','cvsd','cvnni','mean_hr','max_hr','min_hr','std_hr','triangular_index','tinn','total_power','vlf','lf','hf','lf_hf_ratio','lfnu','hfnu','csi','cvi','Modified_csi','sd1','sd2','ratio_sd2_sd1','sampen']
  result_df= pd.DataFrame(columns=column_names) 
  df1=df.rolling(window=windowsizerow)
  # type(df1)
  i=1
  j=0
  for df2 in df1:
    if len(df2)>=windowsizerow and i%int(windowsizerow/2)==0:
      results_td=[]
      results_gd=[]
      results_fd=[]
      results_ccd=[]
      results_ppd=[]
      results_sampen=[]
      results=[]
      rr_corrected= get_rr_corrected_ourdata(df2)   
  

      interpolated_rr_intervals = interpolate_nan_values(rr_corrected,interpolation_method="linear")

      results_td=get_time_domain_features(interpolated_rr_intervals)
      results_gd=get_geometrical_features(interpolated_rr_intervals)
      results_fd=get_frequency_domain_features(interpolated_rr_intervals,"welch",256)
      results_ccd=get_csi_cvi_features(interpolated_rr_intervals)
      results_ppd=get_poincare_plot_features(interpolated_rr_intervals)
      results_sampen=get_sampen(interpolated_rr_intervals)


      results={**results_td,**results_gd,**results_fd,**results_ccd,**results_ppd,**results_sampen}
      # print("in function")
      # print(results)
      result_df=result_df.append(results,ignore_index=True)
    i+=1
    j+=1
  return result_df

## Function for Feature Extraction

In [ ]:
# def baseline_stress_features(input_path=r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants/Part4",output_path=r"/gdrive/MyDrive/SamsungWork/ECGFeatures10Seconds_1May22",partnumber="Part4",windowsize=1280):
#   %cd "$input_path"  
#   # File setup for baseline dataframe
#   baseline_file=['1_ecg_.csv','4_ecg_.csv','7_ecg_.csv','10_ecg_.csv']
#   baseline_df=pd.read_csv(os.path.join(input_path,baseline_file[0]))
#   column_names=['Mean RR (ms)', 'STD RR/SDNN (ms)', "Mean HR (Kubios' style) (beats/min)", 'Mean HR (beats/min)', 'STD HR (beats/min)', 'Min HR (beats/min)', 'Max HR (beats/min)', 'RMSSD (ms)', 'NNxx', 'pNNxx (%)', 'Power VLF (ms2)', 'Power LF (ms2)', 'Power HF (ms2)', 'Power Total (ms2)', 'LF/HF', 'Peak VLF (Hz)', 'Peak LF (Hz)', 'Peak HF (Hz)', 'Fraction LF (nu)', 'Fraction HF (nu)']

#  # File setup for stress dataframe
#   stress_df=pd.read_csv('2_ecg_mathtest.csv')
#   data=pd.read_csv('8_ecg_IQtest.csv')
#   stress_df=stress_df.append(data,ignore_index=True)



#   # removing unnecessary columns
#   baseline_df=baseline_df.drop(['Timestamp','ecg1','PythonTimestamp'],axis=1)
#   stress_df=stress_df.drop(['Timestamp','ecg1','PythonTimestamp'],axis=1)

#   # appending baselinedataframe, adding all data baseline together
#   for base in baseline_file[1:]:
#     data=pd.read_csv(base)
#     baseline_df=baseline_df.append(data,ignore_index=True)

#   labelling_criteria=0

#   # setting up folders for saving
#   if not os.path.exists(os.path.join(output_path,partnumber,"output",str(labelling_criteria))):
#     os.makedirs(os.path.join(output_path,partnumber,"output",str(labelling_criteria)))

#   # baseline feature extraction
#   # print(baseline_df)
#   result_baseline_df=windowing(baseline_df,1280)
#   result_baseline_df['stress_level']=0;
#   result_baseline_df.to_csv(os.path.join(output_path,partnumber,"output",str(labelling_criteria),"baseline.csv"))
#   print(result_baseline_df)

#    # stress feature extraction
#   # %%capture
#   # print(stress_df)
#   result_stress_df=windowing(stress_df,1280)
  

#   result_stress_df['stress_level']=1;
#   result_stress_df.to_csv(os.path.join(output_path,partnumber,"output",str(labelling_criteria),"stress.csv"))
#   print(result_stress_df)
#   # print(partnumber)

#   print("Feature Extracted for %s saved at %s " %(partnumber,os.path.join(output_path,partnumber)))

In [ ]:
def baseline_stress_features(input_path=r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants/Part4",output_path=r"/gdrive/MyDrive/SamsungWork/ECGFeatures10Seconds_1May22",partnumber="Part4",windowsize=1280):
  %cd "$input_path"  
  # File setup for baseline dataframe
  baseline_file=['1_ecg_.csv','4_ecg_.csv','7_ecg_.csv','10_ecg_.csv']
  baseline_df=pd.read_csv(os.path.join(input_path,baseline_file[0]))
  # column_names=['Mean RR (ms)', 'STD RR/SDNN (ms)', "Mean HR (Kubios' style) (beats/min)", 'Mean HR (beats/min)', 'STD HR (beats/min)', 'Min HR (beats/min)', 'Max HR (beats/min)', 'RMSSD (ms)', 'NNxx', 'pNNxx (%)', 'Power VLF (ms2)', 'Power LF (ms2)', 'Power HF (ms2)', 'Power Total (ms2)', 'LF/HF', 'Peak VLF (Hz)', 'Peak LF (Hz)', 'Peak HF (Hz)', 'Fraction LF (nu)', 'Fraction HF (nu)']
  # column_names=['Mean_RR', 'STD_RR_SDNN_Ratio', "Mean_HR_Kubios", 'Mean_HR', 'STD_HR', 'Min_HR', 'Max_HR', 'RMSSD', 'NNxx', 'pNNxx', 'Power_VLF', 'Power_LF', 'Power_HF', 'Power_Total', 'LF/HF', 'Peak_VLF', 'Peak_LF', 'Peak_HF', 'Fraction_LF', 'Fraction_HF']
  column_names=['mean_nni','sdnn','sdsd','rmssd','median_nni','nni_50','pnni_50','nni_20','pnni_20','range_nni','cvsd','cvnni','mean_hr','max_hr','min_hr','std_hr','triangular_index','tinn','total_power','vlf','lf','hf','lf_hf_ratio','lfnu','hfnu','csi','cvi','Modified_csi','sd1','sd2','ratio_sd2_sd1','sampen']


 # File setup for stress dataframe
  stress_df=pd.read_csv('2_ecg_mathtest.csv')
  data=pd.read_csv('8_ecg_IQtest.csv')
  stress_df=stress_df.append(data,ignore_index=True)

 

  # removing unnecessary columns
  baseline_df=baseline_df.drop(['Timestamp','ecg1','PythonTimestamp'],axis=1)
  stress_df=stress_df.drop(['Timestamp','ecg1','PythonTimestamp'],axis=1)

  # appending baselinedataframe, adding all data baseline together
  for base in baseline_file[1:]:
    data=pd.read_csv(base)
    baseline_df=baseline_df.append(data,ignore_index=True)



  # setting up folders for saving
  if not os.path.exists(os.path.join(output_path,partnumber,"output",)):
    os.makedirs(os.path.join(output_path,partnumber,"output"))

  # baseline feature extraction
  # print(baseline_df)
  result_baseline_df=windowing(baseline_df,windowsize)
  result_baseline_df['stress_level']=0;
  result_baseline_df.to_csv(os.path.join(output_path,partnumber,"output","baseline.csv"))
  print(result_baseline_df)

   # stress feature extraction
  # %%capture
  # print(stress_df)
  result_stress_df=windowing(stress_df,windowsize)


  result_stress_df['stress_level']=1;
  result_stress_df.to_csv(os.path.join(output_path,partnumber,"output","stress.csv"))
  print(result_stress_df)
  # print(partnumber)

  print("Feature Extracted for %s saved at %s " %(partnumber,os.path.join(output_path,partnumber)))

In [ ]:
# def baseline_stress_features(input_path=r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants/Part4",output_path=r"/gdrive/MyDrive/SamsungWork/ECGFeatures10Seconds_1May22",partnumber="Part4"):
#   %cd "$input_path"  
#   # File setup for baseline dataframe
#   baseline_file=['1_ecg_.csv','4_ecg_.csv','7_ecg_.csv','10_ecg_.csv']
#   baseline_df=pd.read_csv(os.path.join(input_path,baseline_file[0]))
#   # column_names=['Mean RR (ms)', 'STD RR/SDNN (ms)', "Mean HR (Kubios' style) (beats/min)", 'Mean HR (beats/min)', 'STD HR (beats/min)', 'Min HR (beats/min)', 'Max HR (beats/min)', 'RMSSD (ms)', 'NNxx', 'pNNxx (%)', 'Power VLF (ms2)', 'Power LF (ms2)', 'Power HF (ms2)', 'Power Total (ms2)', 'LF/HF', 'Peak VLF (Hz)', 'Peak LF (Hz)', 'Peak HF (Hz)', 'Fraction LF (nu)', 'Fraction HF (nu)','stress_level']
#   column_names=['Mean_RR', 'STD_RR_SDNN_Ratio', "Mean_HR_Kubios", 'Mean_HR', 'STD_HR', 'Min_HR', 'Max_HR', 'RMSSD', 'NNxx', 'pNNxx', 'Power_VLF', 'Power_LF', 'Power_HF', 'Power_Total', 'LF/HF', 'Peak_VLF', 'Peak_LF', 'Peak_HF', 'Fraction_LF', 'Fraction_HF']
#   result_baseline_df= pd.DataFrame(columns=column_names) 

#  # File setup for stress dataframe
#   stress_df=pd.read_csv('2_ecg_mathtest.csv')
#   data=pd.read_csv('8_ecg_IQtest.csv')
#   stress_df=stress_df.append(data,ignore_index=True)

#   result_stress_df= pd.DataFrame(columns=column_names)

#   # removing unnecessary columns
#   baseline_df=baseline_df.drop(['Timestamp','ecg1','PythonTimestamp'],axis=1)
#   stress_df=stress_df.drop(['Timestamp','ecg1','PythonTimestamp'],axis=1)

#   # appending baselinedataframe, adding all data baseline together
#   for base in baseline_file[1:]:
#     data=pd.read_csv(base)
#     baseline_df=baseline_df.append(data,ignore_index=True)

#   labelling_criteria=0

#   # setting up folders for saving
#   if not os.path.exists(os.path.join(output_path,partnumber,"output",str(labelling_criteria))):
#     os.makedirs(os.path.join(output_path,partnumber,"output",str(labelling_criteria)))

#   # baseline feature extraction
#   print(baseline_df)
#   result_baseline_df=windowing(baseline_df[:15359],result_baseline_df,labelling_criteria)
#   result_baseline_df=windowing(baseline_df[15360:30719],result_baseline_df,labelling_criteria)
#   result_baseline_df=windowing(baseline_df[30720:38408],result_baseline_df,labelling_criteria)
#   result_baseline_df.to_csv(os.path.join(output_path,partnumber,"output",str(labelling_criteria),"baseline.csv"))
#   print(result_baseline_df)

#    # stress feature extraction
#   # %%capture
#   print(stress_df)
#   result_stress_df=windowing(stress_df[:15359],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[15360:30719],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[30720:46079],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[46080:61439],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[61440:76799],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[76780:92139],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[92140:107499],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[107450:120252],result_stress_df,labelling_criteria)

#   result_stress_df.to_csv(os.path.join(output_path,partnumber,"output",str(labelling_criteria),"stress.csv"))
#   print(result_stress_df)
#   # print(partnumber)

#   print("Feature Extracted for %s saved at %s " %(partnumber,os.path.join(output_path,partnumber)))

In [ ]:
# def baseline_stress_features(input_path=r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants/Part4",output_path=r"/gdrive/MyDrive/SamsungWork/ECGFeatures10Seconds_1May22",partnumber="Part4"):
#   %cd "$input_path"  
#   # File setup for baseline dataframe
#   baseline_file=['1_ecg_.csv','4_ecg_.csv','7_ecg_.csv','10_ecg_.csv']
#   baseline_df=pd.read_csv(os.path.join(input_path,baseline_file[0]))
#   column_names=['Mean RR (ms)', 'STD RR/SDNN (ms)', "Mean HR (Kubios' style) (beats/min)", 'Mean HR (beats/min)', 'STD HR (beats/min)', 'Min HR (beats/min)', 'Max HR (beats/min)', 'RMSSD (ms)', 'NNxx', 'pNNxx (%)', 'Power VLF (ms2)', 'Power LF (ms2)', 'Power HF (ms2)', 'Power Total (ms2)', 'LF/HF', 'Peak VLF (Hz)', 'Peak LF (Hz)', 'Peak HF (Hz)', 'Fraction LF (nu)', 'Fraction HF (nu)','stress_level']
#   # column_names=['Mean_RR', 'STD_RR_SDNN_Ratio', "Mean_HR_Kubios", 'Mean_HR', 'STD_HR', 'Min_HR', 'Max_HR', 'RMSSD', 'NNxx', 'pNNxx', 'Power_VLF', 'Power_LF', 'Power_HF', 'Power_Total', 'LF/HF', 'Peak_VLF', 'Peak_LF', 'Peak_HF', 'Fraction_LF', 'Fraction_HF']
#   result_baseline_df= pd.DataFrame(columns=column_names) 

#  # File setup for stress dataframe
#   stress_df=pd.read_csv('2_ecg_mathtest.csv')
#   data=pd.read_csv('8_ecg_IQtest.csv')
#   stress_df=stress_df.append(data,ignore_index=True)

#   result_stress_df= pd.DataFrame(columns=column_names)

#   # removing unnecessary columns
#   baseline_df=baseline_df.drop(['Timestamp','ecg1','PythonTimestamp'],axis=1)
#   stress_df=stress_df.drop(['Timestamp','ecg1','PythonTimestamp'],axis=1)

#   # appending baselinedataframe, adding all data baseline together
#   for base in baseline_file[1:]:
#     data=pd.read_csv(base)
#     baseline_df=baseline_df.append(data,ignore_index=True)

#   labelling_criteria=0

#   # setting up folders for saving
#   if not os.path.exists(os.path.join(output_path,partnumber,"output",str(labelling_criteria))):
#     os.makedirs(os.path.join(output_path,partnumber,"output",str(labelling_criteria)))

#   # baseline feature extraction
#   print(baseline_df)
#   result_baseline_df=windowing(baseline_df[:7679],result_baseline_df,labelling_criteria)
#   result_baseline_df=windowing(baseline_df[7680:15359],result_baseline_df,labelling_criteria)
#   result_baseline_df=windowing(baseline_df[15360:23039],result_baseline_df,labelling_criteria)
#   result_baseline_df=windowing(baseline_df[23040:30719],result_baseline_df,labelling_criteria)
#   result_baseline_df=windowing(baseline_df[30720:38399],result_baseline_df,labelling_criteria)
#   # result_baseline_df=windowing(baseline_df[38400:38408],result_baseline_df,labelling_criteria)
#   result_baseline_df.to_csv(os.path.join(output_path,partnumber,"output",str(labelling_criteria),"baseline.csv"))
#   print(result_baseline_df)

#    # stress feature extraction
#   # %%capture
#   print(stress_df)
#   result_stress_df=windowing(stress_df[:7679],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[7680:15359],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[15360:23039],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[23040:30719],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[30720:38399],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[38400:46079],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[46080:53759],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[53760:61439],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[61440:69119],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[69120:76799],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[76800:84479],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[84480:92159],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[92160:99839],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[99840:107519],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[107520:115199],result_stress_df,labelling_criteria)
#   result_stress_df=windowing(stress_df[115200:120252],result_stress_df,labelling_criteria)

#   result_stress_df.to_csv(os.path.join(output_path,partnumber,"output",str(labelling_criteria),"stress.csv"))
#   print(result_stress_df)
#   # print(partnumber)

#   print("Feature Extracted for %s saved at %s " %(partnumber,os.path.join(output_path,partnumber)))

# Running Function on all Files

In [ ]:
Participants=['Part1','Part2','Part3','Part4','Part5','Part6','Part7','Part8','Part9','Part10','Part11','Part12','Part13','Part14','Part15','Part16','Part17','Part18','Part19','Part20','Part21','Part22','Part23','Part24','Part25','Part26','Part27','Part28','Part29','Part30','Part31','Part32','Part33','Part34','Part35','Part36','Part37','Part38','Part39','Part40','Part41','Part42','Part43','Part44','Part45','Part46','Part47','Part48','Part49','Part50','Part51','Part52','Part53','Part54','Part55','Part56','Part57','Part58','Part59','Part60']

In [ ]:
Working_Participants=['Part2','Part4','Part5','Part8','Part10','Part11','Part12','Part17','Part21','Part23','Part24','Part26','Part27','Part28','Part29','Part30','Part31','Part32','Part33','Part34','Part35','Part36','Part38','Part39','Part40','Part41','Part42','Part43','Part44','Part45','Part46','Part47','Part48','Part49','Part51','Part53','Part54','Part55','Part56','Part57','Part58','Part59','Part60']

In [ ]:
Working_Participants1=['Part4','Part5','Part8','Part10','Part11','Part12','Part17','Part21','Part23','Part24','Part26','Part27','Part28','Part29','Part30','Part31','Part32','Part33','Part34','Part35','Part36','Part38','Part39','Part40','Part41','Part42','Part43','Part44','Part45','Part46','Part47','Part48','Part49','Part51','Part53','Part54','Part55','Part56','Part57','Part58','Part59','Part60']

In [ ]:
Error_Participants=['Part1','Part3','Part6','Part7','Part9','Part13','Part14','Part15','Part16','Part18','Part19','Part20','Part22','Part25','Part37','Part50','Part52']
print(len(Error_Participants))

17


In [ ]:
New_Participants10=['Part4','Part5','Part6','Part10','Part11','Part12','Part14','Part15','Part17','Part23','Part24','Part26','Part27','Part28','Part30','Part31','Part32','Part33','Part34','Part35','Part36','Part38','Part39','Part40','Part42','Part43','Part44','Part45','Part46','Part47','Part48','Part49','Part51','Part53','Part54','Part55','Part56','Part57','Part58','Part59','Part60']

In [ ]:
Error_Participants10=['Part1','Part2','Part3','Part7','Part8','Part9','Part13','Part16','Part18','Part19','Part20','Part21','Part22','Part25','Part29','Part37','Part41','Part50','Part52']

In [ ]:
Participants_5=['Part4','Part5','Part6','Part8','Part10','Part11','Part14','Part15','Part17','Part23','Part24','Part26','Part27','Part28','Part31','Part32','Part33','Part34','Part35','Part36','Part39','Part40','Part42','Part43','Part44','Part45','Part46','Part47','Part48','Part49','Part51','Part52','Part53','Part54','Part55','Part56','Part57','Part58','Part59','Part60']

In [ ]:
ErrorParticipants_5=['Part1','Part2','Part3','Part7','Part12','Part13','Part16','Part18','Part19','Part20','Part21','Part22','Part25','Part29','Part30','Part37','Part38','Part41','Part50']

In [ ]:
# Participants=['Part45','Part46','Part47','Part48','Part49','Part50','Part51','Part52','Part53','Part54','Part55','Part56','Part57','Part58','Part59','Part60']51-33

In [ ]:
# for Part in Error_Participants:
#     baseline_stress_features(os.path.join(r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants", Part,"by_block"),r"/gdrive/MyDrive/SamsungWork/ExtractedFeatures",Part)

In [ ]:
# baseline_stress_features()30->49

In [ ]:
for Part in Participants[3:]:
    baseline_stress_features(os.path.join(r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants", Part,"by_block"),r"/gdrive/MyDrive/SamsungWork/ECGFeatures32_20May22_Final",Part,1280)

/gdrive/.shortcut-targets-by-id/1oRmZOKPo1r153R8uQckeuhxEs5K25BXY/Class_Dataset/CLAS Dataset/Participants/Part4/by_block


/usr/local/lib/python3.7/dist-packages/hrvanalysis/extract_features.py:187: RuntimeWarning: divide by zero encountered in long_scalars
  triang_idx = len(nn_intervals) / max(np.histogram(nn_intervals, bins=range(300, 2000, 8))[0])


        mean_nni         sdnn         sdsd        rmssd  median_nni  nni_50  \
0     432.644509   198.853493   216.816121   216.816203       325.0    76.0   
1     428.303571   191.310349   231.043569   231.044880       325.0    86.0   
2     448.809524   205.700380   234.428212   234.428292       325.0    80.0   
3     385.894737   124.532149   163.818723   163.818723       325.0    84.0   
4     422.500000   178.509158   202.537974   202.538056       325.0    90.0   
5     458.679245   268.175949   335.193749   335.196841       357.5    84.0   
6     582.562500   359.546496   414.626607   414.825252       487.5    78.0   
7     442.544910   188.419527   221.418605   221.419384       357.5    90.0   
8     468.759124   262.961975   302.408664   302.408758       357.5    75.0   
9     483.279221   261.606793   303.913062   303.913136       357.5    88.0   
10    473.480392   231.279556   236.896702   236.897088       357.5    85.0   
11    467.638889   185.384704   221.165235   221.168

ParserError: ignored

# 5 Seconds Window, 2.5 Overlapping

In [ ]:
Participants5=['Part4','Part5','Part6','Part8','Part10','Part11','Part14','Part17','Part24','Part26','Part28','Part31','Part32','Part33','Part34','Part35','Part36','Part39','Part40','Part42','Part43','Part44','Part45','Part46','Part48','Part49','Part51','Part52','Part53','Part54','Part55','Part57','Part58','Part60']

In [ ]:
Error_5=['Part1','Part2','Part3','Part7','Part9','Part12','Part13','Part15','Part16','Part18','Part19','Part20','Part21','Part22','Part23','Part25','Part27','Part29','Part30','Part37','Part38','Part41','Part47','Part50','Part56','Part59']

In [ ]:
# 29->55

In [ ]:
for Part in Participants5[:]:
    baseline_stress_features(os.path.join(r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants", Part,"by_block"),r"/gdrive/MyDrive/SamsungWork/ECGFeatures32_20May22_Final",Part,1280)

/gdrive/.shortcut-targets-by-id/1oRmZOKPo1r153R8uQckeuhxEs5K25BXY/Class_Dataset/CLAS Dataset/Participants/Part4/by_block


/usr/local/lib/python3.7/dist-packages/hrvanalysis/extract_features.py:187: RuntimeWarning: divide by zero encountered in long_scalars
  triang_idx = len(nn_intervals) / max(np.histogram(nn_intervals, bins=range(300, 2000, 8))[0])


        mean_nni         sdnn         sdsd        rmssd  median_nni  nni_50  \
0     432.644509   198.853493   216.816121   216.816203       325.0    76.0   
1     428.303571   191.310349   231.043569   231.044880       325.0    86.0   
2     448.809524   205.700380   234.428212   234.428292       325.0    80.0   
3     385.894737   124.532149   163.818723   163.818723       325.0    84.0   
4     422.500000   178.509158   202.537974   202.538056       325.0    90.0   
5     458.679245   268.175949   335.193749   335.196841       357.5    84.0   
6     582.562500   359.546496   414.626607   414.825252       487.5    78.0   
7     442.544910   188.419527   221.418605   221.419384       357.5    90.0   
8     468.759124   262.961975   302.408664   302.408758       357.5    75.0   
9     483.279221   261.606793   303.913062   303.913136       357.5    88.0   
10    473.480392   231.279556   236.896702   236.897088       357.5    85.0   
11    467.638889   185.384704   221.165235   221.168

/usr/local/lib/python3.7/dist-packages/hrvanalysis/extract_features.py:187: RuntimeWarning: divide by zero encountered in long_scalars
  triang_idx = len(nn_intervals) / max(np.histogram(nn_intervals, bins=range(300, 2000, 8))[0])


       mean_nni         sdnn         sdsd        rmssd  median_nni  nni_50  \
0   2154.285714  1949.134288  2163.937221  2164.986849    1446.250    26.0   
1   5044.464286  3176.600623  3214.086184  3236.471844    4761.250    12.0   
2    683.897849   346.704294   407.197324   407.407048     650.000    75.0   
3    967.008197   713.660724   977.929476   978.181614     812.500    48.0   
4    597.418699   275.379569   364.309372   364.312878     617.500    90.0   
5    687.770270   245.234200   306.447186   306.447756     682.500    94.0   
6    673.387850   269.699180   345.938879   345.955319     650.000    91.0   
7    690.898876   423.338035   502.041850   502.055434     650.000    64.0   
8    582.533482   398.700220   462.883228   462.894432     430.625    75.0   
9   9284.166667  2146.640787  3290.913475  3355.756807   10530.000     4.0   
10  2970.795455  3442.760467  4758.870944  4764.379863    1186.250    18.0   
11  1802.008929  2608.263607  2675.522372  2675.522643     755.6

...

        mean_nni          sdnn          sdsd         rmssd  median_nni  \
0     707.034314    365.311093    488.631085    488.646342     650.000   
1     616.605505    288.881849    378.011237    378.017107     650.000   
2     574.335938    256.655212    288.203237    288.203351     617.500   
3     657.544643    264.102676    318.760059    318.762211     650.000   
4     638.288288    269.744404    306.792238    306.792238     650.000   
5     525.103306    218.380020    245.060770    245.064511     487.500   
6     573.500000    232.672916    292.149877    292.149877     650.000   
7     615.727273    323.021207    339.168316    339.181422     650.000   
8     551.484375    313.589430    301.643893    301.792463     390.000   
9     620.360000    224.561110    247.334242    247.357710     650.000   
10    637.644628    282.932246    322.318521    322.334906     650.000   
11    636.779661    264.551730    308.612736    308.614736     650.000   
12    625.557851    233.656730    284.

/usr/local/lib/python3.7/dist-packages/hrvanalysis/extract_features.py:541: RuntimeWarning: invalid value encountered in sqrt
  sd2 = np.sqrt(2 * np.std(nn_intervals, ddof=1) ** 2 - 0.5 * np.std(diff_nn_intervals, ddof=1) ** 2)
/usr/local/lib/python3.7/dist-packages/hrvanalysis/extract_features.py:187: RuntimeWarning: divide by zero encountered in long_scalars
  triang_idx = len(nn_intervals) / max(np.histogram(nn_intervals, bins=range(300, 2000, 8))[0])


       mean_nni        sdnn        sdsd       rmssd  median_nni  nni_50  \
0    615.774336  311.512382  393.024353  393.035065       650.0    84.0   
1    644.823009  284.476673  324.810163  324.811329       650.0    88.0   
2    614.598214  298.044783  347.716066  347.730982       650.0    78.0   
3    608.333333  245.363910  276.002678  276.002678       650.0    78.0   
4    632.125000  275.509202  277.367683  277.376288       650.0    77.0   
..          ...         ...         ...         ...         ...     ...   
181  654.642857  330.368150  360.310292  360.310411       650.0    85.0   
182  616.099138  317.376009  319.440848  319.455974       650.0    75.0   
183  656.327434  318.666318  387.728958  387.744594       650.0    76.0   
184  625.555556  324.268093  425.985180  425.998447       650.0    79.0   
185  593.760870  332.450097  426.916837  426.917694       552.5    75.0   

       pnni_50  nni_20    pnni_20  range_nni  ...       lfnu       hfnu  \
0    75.000000    96.0  

/usr/local/lib/python3.7/dist-packages/hrvanalysis/extract_features.py:187: RuntimeWarning: divide by zero encountered in long_scalars
  triang_idx = len(nn_intervals) / max(np.histogram(nn_intervals, bins=range(300, 2000, 8))[0])
/usr/local/lib/python3.7/dist-packages/hrvanalysis/extract_features.py:541: RuntimeWarning: invalid value encountered in sqrt
  sd2 = np.sqrt(2 * np.std(nn_intervals, ddof=1) ** 2 - 0.5 * np.std(diff_nn_intervals, ddof=1) ** 2)


       mean_nni        sdnn        sdsd       rmssd  median_nni  nni_50  \
0    743.938356  635.773659  808.327355  808.327481      650.00    54.0   
1    466.683007  225.955724  238.857312  238.868891      357.50    89.0   
2    498.706897  282.066885  324.635864  324.661282      357.50    88.0   
3    557.681159  276.671700  332.349697  332.353845      406.25    91.0   
4    534.300000  267.601089  297.964611  297.967493      487.50    79.0   
..          ...         ...         ...         ...         ...     ...   
181  399.188482  135.829067  190.616190  190.617418      325.00   109.0   
182  377.751256  114.991489  141.983841  141.983936      325.00    86.0   
183  409.285714  149.811016  174.773138  174.784299      325.00    74.0   
184  428.559322  164.424705  199.788421  199.788507      325.00    88.0   
185  404.807692  187.503373  198.260567  198.404068      325.00    76.0   

       pnni_50  nni_20    pnni_20  range_nni  ...       lfnu       hfnu  \
0    75.000000    58.0  

# Combining Features

In [ ]:
PATH = r"/gdrive/MyDrive/SamsungWork/ECGFeatures32_20May22_Final"
EXT = "*.csv"
all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
# len(all_csv_files)
column_names=['mean_nni','sdnn','sdsd','rmssd','median_nni','nni_50','pnni_50','nni_20','pnni_20','range_nni','cvsd','cvnni','mean_hr','max_hr','min_hr','std_hr','triangular_index','tinn','total_power','vlf','lf','hf','lf_hf_ratio','lfnu','hfnu','csi','cvi','Modified_csi','sd1','sd2','ratio_sd2_sd1','sampen']

combineddf= pd.DataFrame(columns=column_names) 


In [ ]:
for featuredata in all_csv_files:
      tempdata=pd.read_csv(featuredata)
      combineddf=combineddf.append(tempdata,ignore_index=True)


In [ ]:
combineddf.to_csv(r"/gdrive/MyDrive/SamsungWork/ECGFeatures32_20May22_Final/CombinedFeatures.csv")


In [ ]:
print(combineddf)

      Mean RR (ms)  STD RR/SDNN (ms)  Mean HR (Kubios' style) (beats/min)  \
0       432.644509        198.277939                           138.681987   
1       428.303571        190.740124                           140.087555   
2       448.809524        205.087262                           133.687003   
3       385.894737        124.204000                           155.482815   
4       422.500000        178.009831                           142.011834   
...            ...               ...                                  ...   
8046    392.606952        138.208996                           152.824599   
8047    392.635135        136.214936                           152.813629   
8048    380.906736        125.434041                           157.518874   
8049    372.920918        117.016407                           160.892020   
8050    389.301075        135.180517                           154.122359   

      Mean HR (beats/min)  STD HR (beats/min)  Min HR (beats/min)  \
0     